In [1]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

### loading the dataset
data=gutenberg.raw('shakespeare-hamlet.txt')

### saving dataset to a file
with open('hamlet.txt','w') as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\Windows\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
### Installing required libraries
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras. preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [3]:
### loading the dataset
with open('hamlet.txt','r') as file:
    text=file.read().lower()

In [4]:
### Text preprocessing
### Tokenizing the text
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index)+1
total_words

4818

In [5]:
### creating input sequences
in_sequences=[]
for line in text.split('\n'):
    token_lst=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_lst)):
        n_gram_sequence=token_lst[:i+1]
        in_sequences.append(n_gram_sequence)



In [6]:
### padding the sequences
max_seq_len=max([len(x) for x in in_sequences])
input_sequences=np.array(pad_sequences(in_sequences,maxlen=max_seq_len,padding='pre'))
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]])

In [7]:
### create predictors and label
x,y=input_sequences[:,:-1],input_sequences[:,-1]

In [8]:
y=tf.keras.utils.to_categorical(y,num_classes=total_words)

In [9]:
### Splitting data into train and validation/test datasets
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [10]:
### Importig required libraries to train a LSTM RNN model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Dense,Dropout,GRU

### Define the mdoel
model=Sequential()
model.add(Embedding(total_words,100,input_length=(max_seq_len-1)))
model.add(GRU(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(100))
model.add(Dense(total_words,activation='softmax'))

### compiling the model
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 13, 100)           481800    
                                                                 
 gru (GRU)                   (None, 13, 150)           113400    
                                                                 
 dropout (Dropout)           (None, 13, 150)           0         
                                                                 
 gru_1 (GRU)                 (None, 100)               75600     
                                                                 
 dense (Dense)               (None, 4818)              486618    
                                                                 
Total params: 1157418 (4.42 MB)
Trainable params: 1157418 (4.42 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
### Train the model
history=model.fit(x_train,y_train,epochs=50,validation_data=(x_test,y_test),verbose=1)

Epoch 1/50


644/644 [==============================] - 44s 53ms/step - loss: 7.0718 - accuracy: 0.0305 - val_loss: 7.0003 - val_accuracy: 0.0348
Epoch 2/50
644/644 [==============================] - 33s 51ms/step - loss: 6.6092 - accuracy: 0.0315 - val_loss: 6.9705 - val_accuracy: 0.0357
Epoch 3/50
644/644 [==============================] - 32s 50ms/step - loss: 6.4228 - accuracy: 0.0433 - val_loss: 6.9348 - val_accuracy: 0.0414
Epoch 4/50
644/644 [==============================] - 32s 50ms/step - loss: 6.2669 - accuracy: 0.0477 - val_loss: 6.9985 - val_accuracy: 0.0478
Epoch 5/50
644/644 [==============================] - 29s 45ms/step - loss: 6.1333 - accuracy: 0.0545 - val_loss: 6.9618 - val_accuracy: 0.0612
Epoch 6/50
644/644 [==============================] - 27s 42ms/step - loss: 5.9353 - accuracy: 0.0698 - val_loss: 6.9646 - val_accuracy: 0.0690
Epoch 7/50
644/644 [==============================] - 28s 43ms/step - loss: 5.7030 - accuracy: 0.0837 - val_loss: 6.9641 - val_accurac

In [12]:
### Function to predict the next word
def predict_next_word(model,tokenizer,text,max_seq_len):
    token_lst=tokenizer.texts_to_sequences([text])[0]
    if len(token_lst)>=max_seq_len:
        token_lst=token_lst[-(max_seq_len-1)]
    token_lst=pad_sequences([token_lst],maxlen=max_seq_len-1,padding='pre')
    predicted=model.predict(token_lst,verbose=0)
    predict_word_index=np.argmax(predicted,axis=1)
    for word,index in tokenizer.word_index.items():
        if index==predict_word_index:
            return word
    return None

In [13]:
input_text='To beat the man you got to be the'
print(f'Input_text:{input_text}')
max_seq_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_seq_len)
print(f'Next word prediction:{next_word}')

Input_text:To beat the man you got to be the
Next word prediction:kinde


In [14]:
### saving the model
model.save('next_word_pred_gru_rnn.h5')

### saving tokenizer
import pickle
with open('tokenizer1.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)

c:\Users\Windows\LLM_course\myevnv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
input_text='To be not to be'
print(f'Input_text:{input_text}')
max_seq_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_seq_len)
print(f'Next word prediction:{next_word}')

Input_text:To be not to be
Next word prediction:buried
